In [8]:
query = '''
CREATE TABLE IF NOT EXISTS BOOKS(
    RANK INTEGER PRIMARY KEY,
    NAME TEXT,
    PRICE INTEGER,
    SALES INTEGER
)
'''

In [9]:
import pymysql
# MySQL 서버에 연결
conn = pymysql.connect(host='localhost', user='root', password='1234',  db='my_db',
 charset='utf8') # 한글처리 (charset = 'utf8’)
# 커서 생성
cursor = conn.cursor()

In [10]:
cursor.execute("CREATE DATABASE IF NOT EXISTS oliveyoung DEFAULT CHARACTER SET utf8mb4")
cursor.execute("USE oliveyoung")  # DB 선택

0

In [ ]:
create_table = '''
CREATE TABLE IF NOT EXISTS oliveyoung (
    id INT AUTO_INCREMENT PRIMARY KEY,
    `RANK` INT,
    NAME VARCHAR(255),
    PRICE VARCHAR(50),
    SALES VARCHAR(50)
)
'''
cursor.execute(create_table)

ins_query = ''' 
INSERT INTO oliveyoung (`RANK`, TITLE, PRICE, SALES) VALUES (%s, %s, %s, %s)
''' 

for book in oliveyoung_list:
    cursor.execute(ins_query, book.to_list())
conn.commit()
 # 연결과 커서 닫기
cursor.close()
conn.close()

# 올리브영 메이크업 데이터 수집 및 저장

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
# ChromeDriverManager를 임포트합니다.
from webdriver_manager.chrome import ChromeDriverManager 

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time # 페이지 로드를 위해 잠시 기다릴 때 사용

In [19]:
# 1. 크롬 드라이버 설정 (webdriver-manager가 자동으로 처리)
# 이 한 줄이 크롬 드라이버를 자동으로 찾아주고, 없으면 다운로드까지 해줍니다.
service = Service(ChromeDriverManager().install()) 

# 2. 웹 드라이버 옵션 설정
# headless 모드: 브라우저 창이 뜨지 않고 백그라운드에서 실행됩니다.
# 창을 직접 보고 싶으면 'options.add_argument('headless')' 줄을 주석 처리하거나 삭제하세요.
options = webdriver.ChromeOptions()
options.add_argument('headless') 
options.add_argument('window-size=1920x1080') # 화면 크기 설정 (일부 웹사이트는 화면 크기에 따라 UI가 달라질 수 있음)
# User-Agent 설정: 일반적인 브라우저처럼 보이게 하여 봇 감지를 회피하는 데 도움을 줍니다.
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

# 3. 웹 드라이버 실행
driver = webdriver.Chrome(service=service, options=options)

# 타겟 URL
url = "https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010002&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EB%A9%94%EC%9D%B4%ED%81%AC%EC%97%85"

print(f"Selenium으로 웹 페이지 로드 중: {url}")
driver.get(url) # 웹페이지 로드

# 4. 페이지 콘텐츠가 완전히 로드될 때까지 명시적으로 기다리기
# 'ul.cate_prd_list > li' 요소 (상품 목록의 첫 번째 아이템)가 페이지에 나타날 때까지 최대 10초 대기합니다.
# 이렇게 하면 JavaScript가 모든 내용을 로드할 시간을 줍니다.
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "ul.cate_prd_list > li"))
    )
    print("상품 목록 요소가 성공적으로 로드되었습니다.")
except Exception as e:
    print(f"상품 목록 로드 대기 중 시간 초과 또는 오류 발생: {e}")
    print("페이지가 완전히 로드되지 않았거나, 셀렉터가 변경되었을 수 있습니다.")
    # 오류 발생 시 현재 로드된 HTML을 출력하여 디버깅에 활용할 수 있습니다:
    # print(driver.page_source) 

# 5. 브라우저가 렌더링한 최종 HTML 콘텐츠 가져오기
# 이제 JavaScript가 실행된 후의 HTML을 가져옵니다.
html_content = driver.page_source

# 6. BeautifulSoup으로 HTML 파싱
soup = BeautifulSoup(html_content, 'html.parser')

# 7. 상품 목록 추출
# 이제 이 'items' 리스트의 순서는 웹 페이지에서 보이는 순서와 일치할 것입니다.
items = soup.select("ul.cate_prd_list > li")

print(f"찾은 상품 아이템 개수: {len(items)}")

products = []
rank_counter = 1 # 순위를 위한 카운터 변수 초기화

for item in items:
    # 순위 정보 추출 (웹페이지에서 보이는 순서대로 1, 2, 3... 부여)
    # Selenium을 사용하면 HTML에 보이는 순서가 페이지 순서와 일치하므로,
    # 굳이 span.thumb_flag.best를 추출하지 않고 순차 번호를 부여하는 것이 더 확실합니다.
    rank = str(rank_counter) 
    
    # 상품명 추출
    name_tag = item.select_one("div.prd_name a")
    name = name_tag.text.strip() if name_tag else None

    # 가격 추출
    price_nums = item.select("p.prd_price span.tx_num")
    original_price = None
    discounted_price = None

    if price_nums:
        if len(price_nums) > 1: # 할인가가 있는 경우 (두 개의 span.tx_num)
            original_price = price_nums[0].text.strip()
            discounted_price = price_nums[1].text.strip()
        else: # 할인가가 없는 경우 (하나의 span.tx_num)
            original_price = price_nums[0].text.strip()
            discounted_price = price_nums[0].text.strip() # 할인가를 정가와 동일하게 설정

    if name: # 상품명이 추출되었을 경우에만 리스트에 추가
        products.append({
            "순위": rank, 
            "상품명": name,
            "정가": original_price,
            "할인가": discounted_price
        })
        rank_counter += 1 # 다음 상품을 위해 순위 카운터 증가

# 8. Pandas DataFrame 생성 및 CSV 저장
df = pd.DataFrame(products)
df.to_csv("oliveyoung_makeup.csv", index=False, encoding="utf-8-sig")
print("✅ 올리브영 메이크업 랭킹 데이터 저장 완료!")

# 9. 웹 드라이버 종료 (매우 중요! 브라우저 프로세스를 닫습니다)
driver.quit()

Selenium으로 웹 페이지 로드 중: https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010002&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EB%A9%94%EC%9D%B4%ED%81%AC%EC%97%85
상품 목록 요소가 성공적으로 로드되었습니다.
찾은 상품 아이템 개수: 100
✅ 올리브영 메이크업 랭킹 데이터 저장 완료!


# SQL에 데이터 넣기

In [24]:
import sqlite3

# SQLite 연결 및 DB 파일 생성 (없으면 자동 생성)
conn = sqlite3.connect("oliveyoung.db")
cursor = conn.cursor()

# 테이블 생성 (없으면 생성)
cursor.execute("""
CREATE TABLE IF NOT EXISTS oliveyoung_makeup_ranking (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    순위 INTEGER,
    상품명 TEXT NOT NULL,
    정가 TEXT,
    할인가 TEXT
)
""")

# 데이터 삽입
insert_query = """
INSERT INTO oliveyoung_makeup_ranking (순위, 상품명, 정가, 할인가)
VALUES (?, ?, ?, ?)
"""

for _, row in df.iterrows():
    values = (
        int(row["순위"]),
        row["상품명"],
        row["정가"],
        row["할인가"]
    )
    cursor.execute(insert_query, values)

# 커밋 및 연결 종료
conn.commit()
conn.close()

print("SQLite 저장 완료: oliveyoung.db, 총", len(df), "개")

SQLite 저장 완료: oliveyoung.db, 총 100 개


In [25]:
import pymysql

# MySQL 연결 설정
conn = pymysql.connect(
    host='localhost',       # 또는 '127.0.0.1'
    user='root',            # 사용자 이름
    password='1234',        # 비밀번호
    database='oliveyoung',  # 위에서 만든 DB 이름
    charset='utf8mb4',
    autocommit=True
)

cursor = conn.cursor()

# 테이블 생성 (없으면 생성)
cursor.execute("""
CREATE TABLE IF NOT EXISTS oliveyoung_makeup_ranking (
    id INT AUTO_INCREMENT PRIMARY KEY,
    순위 INT,
    상품명 VARCHAR(255) NOT NULL,
    정가 VARCHAR(50),
    할인가 VARCHAR(50)
)
""")

# 데이터 삽입
insert_query = """
INSERT INTO oliveyoung_makeup_ranking (순위, 상품명, 정가, 할인가)
VALUES (%s, %s, %s, %s)
"""

for _, row in df.iterrows():
    values = (
        int(row["순위"]),
        row["상품명"],
        row["정가"],
        row["할인가"]
    )
    cursor.execute(insert_query, values)

# 종료
cursor.close()
conn.close()

print("MySQL DB 저장 완료! 총", len(df), "개")

MySQL DB 저장 완료! 총 100 개
